In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("financial_accounting.csv")
df.head()

,Date,Account,Description,Debit,Credit,Category,Transaction_Type,Customer_Vendor,Payment_Method,Reference
0,2023-08-21,Accounts Payable,Transaction 1,112.56,112.56,Asset,Sale,Customer 39,Cash,67471
1,2023-08-13,Accounts Receivable,Transaction 2,775.86,775.86,Revenue,Purchase,Customer 3,Check,92688
2,2023-05-11,Accounts Receivable,Transaction 3,332.81,332.81,Revenue,Transfer,Customer 36,Check,72066
3,2023-02-26,Accounts Receivable,Transaction 4,203.71,203.71,Asset,Purchase,Customer 57,Check,27973
4,2023-11-06,Accounts Receivable,Transaction 5,986.26,986.26,Asset,Expense,Customer 92,Check,29758


In [3]:
class AccountingAnalytics:
    def __init__(self, df: pd.DataFrame):
        # Ensure date is datetime
        df["Date"] = pd.to_datetime(df["Date"])
        self.df = df.copy()
    
    # -----------------------------
    # 1. Trial Balance
    # -----------------------------
    def trial_balance(self):
        tb = self.df.groupby("Account")[["Debit", "Credit"]].sum().reset_index()
        tb["Balance"] = tb["Debit"] - tb["Credit"]
        return tb
    
    # -----------------------------
    # 2. Income Statement
    # -----------------------------
    def income_statement(self):
        revenue = self.df[self.df["Category"] == "Revenue"]["Credit"].sum()
        expenses = self.df[self.df["Category"] == "Expense"]["Debit"].sum()
        net_profit = revenue - expenses
        return pd.DataFrame({
            "Category": ["Revenue", "Expenses", "Net Profit"],
            "Amount": [revenue, expenses, net_profit]
        })
    
    # -----------------------------
    # 3. Balance Sheet
    # -----------------------------
    def balance_sheet(self):
        assets = self.df[self.df["Category"] == "Asset"]
        liabilities = self.df[self.df["Category"] == "Liability"]

        total_assets = assets["Debit"].sum() - assets["Credit"].sum()
        total_liabilities = liabilities["Credit"].sum() - liabilities["Debit"].sum()
        equity = total_assets - total_liabilities

        return pd.DataFrame({
            "Category": ["Assets", "Liabilities", "Equity"],
            "Amount": [total_assets, total_liabilities, equity]
        })
    
    # -----------------------------
    # 4. Cash Flow Statement (simplified)
    # -----------------------------
    def cash_flow(self):
        cash_txn = self.df[self.df["Payment_Method"] == "Cash"]
        inflows = cash_txn["Debit"].sum()
        outflows = cash_txn["Credit"].sum()
        net_cash = inflows - outflows
        return pd.DataFrame({
            "Category": ["Cash Inflows", "Cash Outflows", "Net Cash Flow"],
            "Amount": [inflows, outflows, net_cash]
        })
    
    # -----------------------------
    # 5. Aging Report
    # -----------------------------
    def aging_report(self, account_name="Accounts Receivable"):
        today = pd.Timestamp.today()
        ar = self.df[self.df["Account"] == account_name].copy()
        if ar.empty:
            return pd.DataFrame({"Aging_Bucket": [], "Amount": []})
        
        ar["Days_Outstanding"] = (today - ar["Date"]).dt.days
        bins = [0, 30, 60, 90, 120, 9999]
        labels = ["0-30", "31-60", "61-90", "91-120", "120+"]
        ar["Aging_Bucket"] = pd.cut(ar["Days_Outstanding"], bins=bins, labels=labels, right=True)
        return ar.groupby("Aging_Bucket")["Debit"].sum().reset_index()
    
    # -----------------------------
    # 6. Transaction Drill-down
    # -----------------------------
    def drill_down(self, account=None, customer=None, txn_type=None):
        filtered = self.df.copy()
        if account:
            filtered = filtered[filtered["Account"] == account]
        if customer:
            filtered = filtered[filtered["Customer_Vendor"] == customer]
        if txn_type:
            filtered = filtered[filtered["Transaction_Type"] == txn_type]
        return filtered
    
    # -----------------------------
    # 7. Error Checks
    # -----------------------------
    def error_checks(self):
        errors = {}

        # Trial balance check
        if abs(self.df["Debit"].sum() - self.df["Credit"].sum()) < 1e-6:
            errors["Trial_Balance"] = "✅ Balanced"
        else:
            errors["Trial_Balance"] = "❌ Not Balanced"
        
        # Unbalanced transactions
        self.df["Balanced"] = self.df["Debit"].round(2) == self.df["Credit"].round(2)
        unbalanced = self.df[~self.df["Balanced"]]
        errors["Unbalanced_Entries"] = unbalanced

        # Anomalies (negative values, missing categories)
        anomalies = self.df[(self.df["Debit"] < 0) | (self.df["Credit"] < 0) | (self.df["Category"].isna())]
        errors["Anomalies"] = anomalies
        
        return errors


In [4]:
analytics = AccountingAnalytics(df)

print("Trial Balance")
display(analytics.trial_balance())

print("Income Statement")
display(analytics.income_statement())

print("Balance Sheet")
display(analytics.balance_sheet())

print("Cash Flow")
display(analytics.cash_flow())

print("Aging Report (AR)")
display(analytics.aging_report("Accounts Receivable"))

print("Transaction Drill-Down (Customer 46)")
display(analytics.drill_down(customer="Customer 46"))

print("Error Checks")
errors = analytics.error_checks()
print(errors["Trial_Balance"])
display(errors["Unbalanced_Entries"])
display(errors["Anomalies"])


Trial Balance


,Account,Debit,Credit,Balance
0,Accounts Payable,15152628.73,15152628.73,0.0
1,Accounts Receivable,15272312.94,15272312.94,0.0
2,Cash,15067137.91,15067137.91,0.0
3,Inventory,15282657.25,15282657.25,0.0


Income Statement


,Category,Amount
0,Revenue,15268361.16
1,Expenses,15163910.14
2,Net Profit,104451.02


Balance Sheet


...

Transaction Drill-Down (Customer 46)


,Date,Account,Description,Debit,Credit,Category,Transaction_Type,Customer_Vendor,Payment_Method,Reference
34,2023-02-28,Inventory,Transaction 35,376.11,376.11,Expense,Transfer,Customer 46,Check,25242
41,2023-11-15,Inventory,Transaction 42,975.01,975.01,Expense,Expense,Customer 46,Bank Transfer,92447
72,2023-07-03,Inventory,Transaction 73,683.34,683.34,Liability,Expense,Customer 46,Check,75672
132,2023-12-07,Accounts Receivable,Transaction 133,443.97,443.97,Revenue,Sale,Customer 46,Credit Card,46517
151,2023-02-26,Accounts Payable,Transaction 152,604.83,604.83,Liability,Sale,Customer 46,Credit Card,44713
...,...,...,...,...,...,...,...,...,...,...
99499,2023-10-21,Accounts Payable,Transaction 99500,852.12,852.12,Asset,Expense,Customer 46,Credit Card,51035
99619,2023-08-19,Accounts Payable,Transaction 99620,614.46,614.46,Liability,Expense,Customer 46,Cash,41025
99753,2023-12-07,Inventory,Transaction 99754,409.26,409.26,Liability,Expense,Customer 46,Check,17501
99765,2023-09-18,Accounts Receivable,Transaction 99766,750.20,750.20,Asset,Transfer,Customer 46,Credit Card,40712


Error Checks
✅ Balanced


,Date,Account,Description,Debit,Credit,Category,Transaction_Type,Customer_Vendor,Payment_Method,Reference,Balanced


,Date,Account,Description,Debit,Credit,Category,Transaction_Type,Customer_Vendor,Payment_Method,Reference,Balanced


In [5]:
# Export the transactions DataFrame to CSV
df.to_csv("financial_accounting2.csv", index=False)
